In [1]:
#Uso de las primeras 10 capas de VGG16, añadiendo una CNN.

In [2]:
import time
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(72, 96, 3))

Using TensorFlow backend.
/home/rauly/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/rauly/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 72, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 72, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 72, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 36, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 36, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 36, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 18, 24, 128)       0         
__________

In [4]:
for i in range (1,10):
    conv_base.layers.pop()

In [5]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 72, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 72, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 72, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 36, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 36, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 36, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 18, 24, 128)       0         
__________

In [6]:
inp = conv_base.input
out =conv_base.layers[-1].output

In [7]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 72, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 72, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 72, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 36, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 36, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 36, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 18, 24, 128)       0         
__________

In [8]:
from keras.models import Model
model2 = Model(inp, out)

In [9]:
cont = 0
for layer in model2.layers:
    cont = cont + 1
    if (cont >= 8):
        layer.trainable = True
    else:
        layer.trainable = False

In [10]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 72, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 72, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 72, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 36, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 36, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 36, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 18, 24, 128)       0         
__________

In [11]:
#Copying images to training, validation, and test directories
import os, shutil
import numpy as np
#Path where my dataset is stored
base_dir = '/home/rauly/Documents/Augmentation/Glasses16/Thermal_Glasses16/Training_Sections'
base_dir1 = '/home/rauly/Documents/Augmentation/Glasses16/Thermal_Glasses16/Testing_Sections'

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir1, 'test')

In [12]:
from keras import layers
from keras import models
from keras import regularizers
from keras.layers.normalization import BatchNormalization

model = models.Sequential()
model.add(model2)
model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(256, (3, 3), activation='relu'))
#model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 18, 24, 256)       1735488   
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 12, 256)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 12, 256)        1024      
_________________________________________________________________
flatten_1 (Flatten)          (None, 27648)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                442384    
Total params: 2,178,896
Trainable params: 1,918,224
Non-trainable params: 260,672
_________________________________________________________________


In [13]:
from keras import metrics
import functools
from functools import partial
top2_acc = functools.partial(metrics.top_k_categorical_accuracy, k=2)
top3_acc = functools.partial(metrics.top_k_categorical_accuracy, k=3)
top2_acc.__name__ = 'top2_acc'
top3_acc.__name__ = 'top3_acc'

In [14]:
#Compile stage
from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4), #Decrease learning rate
              metrics=['accuracy',top2_acc, top3_acc])

In [15]:
#Using ImageDataGenerator to read images from directories
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(72, 96),
        batch_size= 4, #20
        color_mode='rgb',
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')
        #save_to_dir='/home/rauly/Documents/Augmentation/Augmented_images2')
        #save_prefix='perf',
        #save_format='jpg')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(72, 96),
        batch_size=74,
        color_mode='rgb',
        class_mode='categorical')

Found 1168 images belonging to 16 classes.
Found 1184 images belonging to 16 classes.


In [21]:
#Previously trained, but I have to restart the process because of kernel's issues
#Fitting the model using a batch generator
history = model.fit_generator(
      train_generator,
      steps_per_epoch=292, #70
      epochs= 10,
      validation_data=validation_generator,
      validation_steps=16,
      workers=0,
      max_queue_size=0)

Epoch 1/10
292/292 [==============================] - 97s 334ms/step - loss: 0.1725 - acc: 0.9572 - top2_acc: 0.9752 - top3_acc: 0.9786 - top4_acc: 0.9803 - top5_acc: 0.9812 - val_loss: 0.5156 - val_acc: 0.8666 - val_top2_acc: 0.9350 - val_top3_acc: 0.9628 - val_top4_acc: 0.9797 - val_top5_acc: 0.9924
Epoch 2/10
292/292 [==============================] - 99s 338ms/step - loss: 3.6667e-04 - acc: 1.0000 - top2_acc: 1.0000 - top3_acc: 1.0000 - top4_acc: 1.0000 - top5_acc: 1.0000 - val_loss: 0.0871 - val_acc: 0.9713 - val_top2_acc: 0.9932 - val_top3_acc: 0.9983 - val_top4_acc: 1.0000 - val_top5_acc: 1.0000
Epoch 3/10
292/292 [==============================] - 98s 337ms/step - loss: 8.3951e-06 - acc: 1.0000 - top2_acc: 1.0000 - top3_acc: 1.0000 - top4_acc: 1.0000 - top5_acc: 1.0000 - val_loss: 0.0322 - val_acc: 0.9941 - val_top2_acc: 0.9941 - val_top3_acc: 0.9975 - val_top4_acc: 1.0000 - val_top5_acc: 1.0000
Epoch 4/10
292/292 [==============================] - 99s 337ms/step - loss: 1.2089

In [22]:
#Retraining
model = models.Sequential()
model.add(model2)
#model.add(layers.BatchNormalization())
#model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='softmax'))

In [23]:
#Compile stage
from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4), #Decrease learning rate
              metrics=['accuracy',top2_acc, top3_acc, top4_acc, top5_acc])

In [16]:
#Fitting the model using a batch generator
history = model.fit_generator(
      train_generator,
      steps_per_epoch=292,
      epochs=3,
      workers=0,
      max_queue_size=0)

Epoch 1/3
292/292 [==============================] - 72s 247ms/step - loss: 0.2791 - acc: 0.9307 - top2_acc: 0.9580 - top3_acc: 0.9675
Epoch 2/3
292/292 [==============================] - 65s 222ms/step - loss: 2.2155e-04 - acc: 1.0000 - top2_acc: 1.0000 - top3_acc: 1.0000
Epoch 3/3
292/292 [==============================] - 67s 231ms/step - loss: 1.5812e-06 - acc: 1.0000 - top2_acc: 1.0000 - top3_acc: 1.0000


In [17]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(72, 96),
        batch_size=74,
        color_mode='rgb',
        class_mode='categorical')
start_time = time.time()
test_loss, test_acc, test_top2_acc, test_top3_acc = model.evaluate_generator(test_generator, steps=16, workers=0, max_queue_size=0)
end_time = time.time()
testing_time = end_time - start_time
print('test acc:', test_acc)
print('Testing time:', testing_time)
print('Recognition rate top2: ', test_top2_acc)
print('Recognition rate top3: ', test_top3_acc)

Found 1184 images belonging to 16 classes.
test acc: 0.9569256752729416
Testing time: 32.994813680648804
Recognition rate top2:  0.9864864945411682
Recognition rate top3:  0.9881756827235222


In [18]:
testing_time / 1184

0.02786724128433176

In [26]:
#Test the model with the Glasses Hat dataset

In [27]:
#Path where my dataset is stored
base_dir2 = '/home/rauly/Documents/Augmentation/Glasses16/Thermal_GlassesHat16/Testing_Sections'
# Directories for test splits
test_dir1 = os.path.join(base_dir2, 'test148')

In [28]:
#Using ImageDataGenerator to read images from directories
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir1,
    target_size=(72, 96),
    batch_size=148,
    color_mode='rgb',
    class_mode='categorical')

Found 2368 images belonging to 16 classes.


In [29]:
test_loss, test_acc, test_top2_acc, test_top3_acc, test_top4_acc, test_top5_acc = model.evaluate_generator(test_generator, steps=16, workers=0, max_queue_size=0)
print('Recognition rate top1: ', test_acc)
print('Recognition rate top2: ', test_top2_acc)
print('Recognition rate top3: ', test_top3_acc)
print('Recognition rate top4: ', test_top4_acc)
print('Recognition rate top5: ', test_top5_acc)

Recognition rate top1:  0.9493243247270584
Recognition rate top2:  0.9687499962747097
Recognition rate top3:  0.9805743210017681
Recognition rate top4:  0.985219594091177
Recognition rate top5:  0.9919763430953026


In [30]:
#Test the model with the All dataset

In [31]:
#Path where my dataset is stored
base_dir3 = '/home/rauly/Documents/Augmentation/Glasses16/Thermal_All16/Testing_Sections'
# Directories for test splits
test_dir2 = os.path.join(base_dir3, 'test148')

In [32]:
#Using ImageDataGenerator to read images from directories
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir2,
    target_size=(72, 96),
    batch_size=148,
    color_mode='rgb',
    class_mode='categorical')

Found 2368 images belonging to 16 classes.


In [33]:
test_loss, test_acc, test_top2_acc, test_top3_acc, test_top4_acc, test_top5_acc = model.evaluate_generator(test_generator, steps=16, workers=0, max_queue_size=0)
print('Recognition rate top1: ', test_acc)
print('Recognition rate top2: ', test_top2_acc)
print('Recognition rate top3: ', test_top3_acc)
print('Recognition rate top4: ', test_top4_acc)
print('Recognition rate top5: ', test_top5_acc)

Recognition rate top1:  0.9108952693641186
Recognition rate top2:  0.9362331070005894
Recognition rate top3:  0.965371623635292
Recognition rate top4:  0.9784628376364708
Recognition rate top5:  0.9907094538211823
